### This Notebook provides some Notes about CNN using PyTorch for custom DataSet

check data on Kaggle : https://www.kaggle.com/c/arabic-hwr-ai-pro-intake1/data

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image

import math
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as f
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torchvision

from sklearn.model_selection import train_test_split

In [3]:
## device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Load and prepare The Dataset

In [4]:
## Hyperparameters
learning_rate = 0.001
n_epochs = 100
n_batch_size = 32

In [5]:
## Note that these paths is on kaggle --> check the above link for data

train_labels = pd.read_csv('../input/arabic-hwr-ai-pro-intake1/train.csv')
train_images = Path(r'../input/arabic-hwr-ai-pro-intake1/train')

## read these all training images paths as Series
train_images_paths = pd.Series(sorted(list(train_images.glob(r'*.png'))), name='Filepath').astype(str)

train_images_paths.head()

#### Explore the Data

In [6]:
img_key_value = {}
for value in train_labels['label'].unique():
    img_key_value[value] = train_labels[train_labels['label']==value].index[0]
    
img_index = list(img_key_value.values())
img_label = list(img_key_value.keys())

fig, ax = plt.subplots(4, 7, figsize=(12, 8))

i = 0
for row in range(4):
    for col in range(7):
        plt.sca(ax[row, col])
        plt.title(f'label = {img_label[i]}')
        img = plt.imread(train_images_paths.iloc[img_index[i]])
        plt.imshow(img)
        plt.axis('off')
        i+=1

In [7]:
print('Number of Instances in train_set =>', len(train_images_paths))
print('Number of Instances in train_labels =>', len(train_labels))
img = plt.imread(train_images_paths.iloc[img_index[0]])
print('shape of each Image is =>', img.shape)
print()

train_full_labels = train_labels['label'].values
train_full_set = np.empty((13440, 32, 32, 3), dtype=np.float32)  #take only the first 3 channels

for idx, path in enumerate(train_images_paths):
    img = plt.imread(path)
    img = img[:,:,:3]
    train_full_set[idx] = img
print()
print('train_full_set.shape =>', train_full_set.shape)
print('train_full_labels.shape =>', train_full_labels.shape)
print()

## for labels
train_full_labels = (train_labels['label'] - 1).values  ## to start from 0
print('train_full_labels.shape =>', train_full_labels.shape)


## split the Dataset
X_train, X_valid, y_train, y_valid = train_test_split(train_full_set, train_full_labels, 
                                                      test_size=0.2, shuffle=True, random_state=42)

### to float32 then to tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_valid = torch.from_numpy(X_valid.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_valid = torch.from_numpy(y_valid.astype(np.float32))
## reshape
y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)

## PyTorch want to get the Channels first
X_train = torch.permute(X_train, (0, 3, 1, 2))
X_valid = torch.permute(X_valid, (0, 3, 1, 2))
##  for target  --> this is a MUST
y_train = y_train.type(torch.LongTensor)
y_valid = y_valid.type(torch.LongTensor)

print()
print('X_train.shape =>', X_train.shape)
print('X_valid.shape =>', X_valid.shape)
print('y_train.shape =>', y_train.shape)
print('y_valid.shape =>', y_valid.shape)
print()

### Create the DataSet and DataLoader

In [8]:
## create two classes one for train and other for validation

class TrainingDataSet(Dataset):
    def __init__(self, transform=None):
        # get X (features) and y(taregt) ---> to tensors
        self.features = X_train
        self.target = y_train
        
        self.n_samples = X_train.shape[0]
        
    def __getitem__(self, index):
        ## data indexing 
        return self.features[index], torch.squeeze(self.target[index])
        
    def __len__(self):
        ## data length
        return self.n_samples
    
############################ --------------------------------------- ############################

class ValidationDataSet(Dataset):
    def __init__(self, transform=None):
        # get X (features) and y(taregt) ---> to tensors
        self.features = X_valid
        self.target = y_valid
        
        self.n_samples = X_valid.shape[0]
        
    def __getitem__(self, index):
        ## data indexing 
        return self.features[index], torch.squeeze(self.target[index])
        
    def __len__(self):
        ## data length
        return self.n_samples

In [9]:
train_dataset = TrainingDataSet()
valid_dataset = ValidationDataSet()

## create the dataloader
train_loader = DataLoader(dataset=train_dataset, batch_size=n_batch_size, 
                          shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=n_batch_size, 
                          shuffle=False, num_workers=0)

#### Build the Model

In [11]:
input_size = X_train.shape[1]
hidden_size = {'512':512, '256':256, '128':128, '64':64, '32':32}
num_classes = len(np.unique(y_train))
num_classes

In [12]:
## 
class ConvNeuralNetwork(nn.Module):
    def __init__(self):
        super(ConvNeuralNetwork, self).__init__()
        
        ## conv_base  ---> images input 32*32*1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding='same')
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding='same')
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding='same')
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        ## top_clf
        ## made flatten by yourself ---> 64 is the previuos channels --> 6x6 is window at this step
        self.fc1 = nn.Linear(in_features=64*8*8, out_features=256)   ## 256 neurons
        self.fc2 = nn.Linear(in_features=256, out_features=256)       ## 256 neurons
        self.fc3 = nn.Linear(in_features=256, out_features=28)   ## 28 classes
        
        
    def forward(self, x):
        out = f.relu(self.conv1(x))
        out = self.pool1(out)
        out = f.relu(self.conv2(out))
        out = f.relu(self.conv3(out))
        out = self.pool2(out)
        
        ## Flatten
        out = out.view(-1, 64*8*8)  
        out = f.relu(self.fc1(out))
        out = f.relu(self.fc2(out))
        out = self.fc3(out)   ## for softmax --> No Activation function at the end
        return out

In [13]:
## Define the Model
model = ConvNeuralNetwork().to(device=device)

## Criteria
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


## Training
print('Training Started _____________________________________ \n')

n_iterations = len(train_loader)    ## --> = len(train_dataset)/n_batch_size
n_train_samples = 0
n_train_correct = 0

for epoch in range(n_epochs):
    print(f'## epoch No. {epoch+1}')
    for i, (train_images, train_labels) in enumerate(train_loader):
        
        ## images shape ---> batch_size * channels * height * width (32, 3, 32, 32)
        train_images = train_images.to(device=device)
        train_labels = train_labels.to(device=device)
        
        ## forward pass
        y_train_pred = model(train_images)
        ## loss
        l = loss(y_train_pred, train_labels)
        
        ## we can empty gradients first
        optimizer.zero_grad()
        ## backward
        l.backward()
        ## step
        optimizer.step()
        
        ## during training
        ## return value and index --> i care about index of max score
        _, y_train_pred_cls = torch.max(y_train_pred, dim=1)
        
        ## modify
        n_train_samples += train_labels.shape[0]
        n_train_correct += (y_train_pred_cls==train_labels).sum().item()
        train_accuracy = (n_train_correct/n_train_samples) * 100.0
        
        print(f'## Iteration No. {i+1}, loss={l:.5f}, acc={train_accuracy:.3f}')
        
print('Training is finished ___________________________')        

#### Evaluation

In [14]:
## to prevent (requires_grad=True) during evaluation
with torch.no_grad():    
    n_valid_correct = 0
    n_valid_samples = 0
    
    for i, (images_valid, valid_labels) in enumerate(valid_loader):
         ## images shape ---> batch_size * channels * height * width (32, 3, 32, 32)
        images_valid = images_valid.to(device=device)
        valid_labels = valid_labels.to(device=device)
        
        y_valid_pred = model(images_valid)
        
        ## return value and index --> i care about index of max score
        _, y_valid_pred_cls = torch.max(y_valid_pred, dim=1)
        
        ## modify
        n_valid_samples += valid_labels.shape[0]
        n_valid_correct += (y_valid_pred_cls==valid_labels).sum().item()
                
valid_accuracy = (n_valid_correct/n_valid_samples) * 100.0
print(f'Validation accuracy is --> {valid_accuracy:.4f} %')

In [19]:
## total accuracy
n_correct = n_train_correct + n_valid_correct
n_samples = n_train_samples + n_valid_samples

total_acc = (n_correct / n_samples) * 100
print(f'Total Accuracy for full Training Data --> {total_acc:.3f}')

#### Done !